In [0]:
!pip install keras_metrics
!pip install extra-keras-metrics
!pip install keras-self-attention
!pip install -U gensim

  Created wheel for extra-keras-metrics: filename=extra_keras_metrics-1.3.1-cp36-none-any.whl size=13091 sha256=75bf083ffe09bc6f99c2ddaa569006f8250e98ae4c75f45c817f3af818ebed50
  Stored in directory: /root/.cache/pip/wheels/48/f1/b4/a9f81578416b99274b6d5b34fdc92d9bd82b6be99efa965b17
Successfully built extra-keras-metrics
  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=f8fa0371485d195b8ab21172a95c11f15978e07845aeb33821810864cb2b6629
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention
     |████████████████████████████████| 24.2MB 73.6MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, LSTM, Dense, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import Adam
from keras.layers import Flatten
import keras
from keras import backend as K
import tensorflow as tf
import datetime
from tensorboardcolab import *
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau, EarlyStopping
from keras.layers.normalization import BatchNormalization
import keras_metrics as km
from keras import optimizers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from sklearn import metrics
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from sklearn import metrics
from sklearn.utils import class_weight
from keras.layers import Input,Bidirectional
from keras.layers import LSTM,Input, dot, Lambda, Dense, Dropout,CuDNNLSTM, BatchNormalization
from keras.models import Sequential
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers import Dot
from sklearn.metrics import label_ranking_average_precision_score

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Predictions of all answers

In [0]:
 from keras.models import model_from_json
# load json and create model
json_file = open('/content/drive/My Drive/seach/maybefinal/final_model50.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/seach/maybefinal/final_model50.h5")
print("Loaded model from disk")





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Loaded model from disk


# Search Engine:
Where a question is passed and top 5 answers depending on prediction have been sorted

In [0]:
def final_fun_3(question):
  df_csv = pd.read_csv('/content/drive/My Drive/seach/data.tsv', names=['ansindex', 'question', 'answer','anslabel','ans_no'],sep='\t')
  stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
                    "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his','himself', \
                    'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them','their',\
                    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll",'these', 'those', \
                    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having','do', 'does', \
                    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
                    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during','before', 'after',\
                    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under','again', 'further',\
                    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
                    'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
                    's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
                    've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
                    "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',"mightn't", 'mustn',\
                    "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',\
                    "wasn't", 'weren', "weren't",'won', "won't", 'wouldn', "wouldn't"])
  def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
        # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
  
  
  def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace("'", "")
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split())
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text
  
  
  def preprocess_text1(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace("'", "")
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text
  
  
  
  preprocessed_q= preprocess_text(df_csv['question'].values)
  preprocessed_a= preprocess_text1(df_csv['answer'].values)
  df_csv['clean_answer']=preprocessed_a
  df_csv['clean_question']=preprocessed_q
  df_csv.drop(['answer','question'],axis=1,inplace=True)
  df_csv.sort_values(by=['clean_question'])
  
  
  #input
  q=preprocess_text(question)
  
  
  
  answer=[]
  #question=['average number of lightning strikes per day']
  #q=preprocess_text(question)
  for index,row in df_csv.iterrows():
    if(row["clean_question"]==q[0]):
      #print(len(answer))
      answer.append(row["clean_answer"])
      
          
  
  questions = np.tile(question, (len(answer), 1)) 
  flat_ques = [item for sublist in questions for item in sublist]  
  
  
  #, Word2Vec

#currently using gensim's default word2vec pretrained model, trained on googlenews data. 
#todo: train on wikiqa if the results are crap. 

  w2vmodel = KeyedVectors.load_word2vec_format('/content/drive/My Drive/vector/GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)
  
  wvdict = w2vmodel.wv.vocab
  
  vocab_size = len(w2vmodel.index2word) + 1
  
  
  #make embedding matrix
  embedding_matrix = np.zeros((vocab_size, 300))
  for i in range(len(wvdict)):
    embedding_vector = w2vmodel.wv[w2vmodel.wv.index2word[i]]
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector
      
      
  def SEQ_token_and_pad(sequences, max_seq_length):
    encoded_seqs = []
    for qa in sequences:
      encoded_qa = []
      tokenized_qa = qa.split()
      for word in tokenized_qa:
        index_to_append = vocab_size - 1
      
        if word in wvdict.keys():
          index_to_append = w2vmodel.vocab[word].index
        encoded_qa.append(index_to_append)
    
      encoded_seqs.append(encoded_qa)
    padded_seqs = pad_sequences(encoded_seqs, maxlen = max_seq_length, padding="post", value = vocab_size - 1)
    return padded_seqs      
  
  max_hidden_length=161
  
  
  padded_test_questions1 = SEQ_token_and_pad(flat_ques, max_hidden_length)
  padded_test_answers1 = SEQ_token_and_pad(answer, max_hidden_length)
  prob=loaded_model.predict_on_batch([padded_test_questions1, padded_test_answers1])
  p=[el[1] for el in prob]
  df = pd.DataFrame(answer,columns =['Answer'])
  df['prob']=p
  df=df.sort_values(by=['prob'],ascending=False)
  print("top 5 high predicted answers")
  df=df.head(5)
  return df

  
  
  
  
   

  
  

In [0]:
question=['symptoms of a dying mouse']
df=final_fun_3(question)
df
  

100%|██████████| 1/1 [00:00<00:00, 2786.91it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


top 5 high predicted answers


,Answer,prob
6,symptoms similar mouse much worse condition 1 ...,0.909723
2,symptoms dog cat poisoning symptoms poisoned p...,0.909513
5,symptoms similar mouse much worse condition ru...,0.890835
1,symptoms mites include 1 excessive scratching ...,0.747989
7,depending poison ingested poisoned dog cat gas...,0.745571
